# Abstract

Make the test comparison associations

# Environment

In [ ]:
# Make all associations
MAKE_ALL = False

In [ ]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [ ]:
from os import listdir, path
import pdb
import re
from tempfile import TemporaryDirectory

In [ ]:
import numpy as np

In [ ]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [ ]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

# Library

In [ ]:
def make_standards(pool_root, main_args=None):
    """Make the association standards for a pool"""
    pool_path = path.join(
        '..',
        'data',
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join('..', 'data', pool_root + asn_tail)
            %mv $from_path $to_path


# Main

## Level 2 Associations

In [ ]:
lv2_only_args = [
    '-r',
    '../../lib/rules_level2b.py',
    '--ignore-default',
]

In [ ]:
lv3_only_args = [
    '-r',
    '../../lib/rules_level3.py',
    '--ignore-default',
]

In [ ]:
def_args = []

### pool_002_image_miri

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_002_image_miri', main_args=lv3_only_args)

### pool_009_spec_miri_lv2bkg

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_009_spec_miri_lv2bkg', main_args=lv2_only_args)

### pool_010_spec_nirspec_lv2bkg

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_010_spec_nirspec_lv2bkg', main_args=lv2_only_args)

### pool_011_spec_miri_lv2bkg_lrs

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_011_spec_miri_lv2bkg_lrs', main_args=lv2_only_args)

### pool_013_coron_nircam

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_013_coron_nircam', main_args=def_args)

### pool_014_ami_niriss

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_014_ami_niriss', main_args=def_args)

### pool_015_spec_nirspec_lv2bkg_reversed

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_015_spec_nirspec_lv2bkg_reversed', main_args=lv2_only_args)

### pool_016_spec_nirspec_lv2bkg_double

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_016_spec_nirspec_lv2bkg_double', main_args=lv2_only_args)

### pool_017_spec_nirspec_lv2imprint

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_017_spec_nirspec_lv2imprint', main_args=lv2_only_args)

### pool_018_all_exptypes

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_018_all_exptypes', main_args=lv2_only_args)

### pool_019_niriss_wfss

In [ ]:
make_me = MAKE_ALL
if make_me:
    make_standards('pool_019_niriss_wfss', main_args=def_args)